In [1]:
# Following tutorial found on http://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [5]:
# importing libraries
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
# load dataset
df = pandas.read_csv("housing.data.csv", delim_whitespace=True, header=None)
dataset = df.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]

# Training a Baseline model

In [10]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [11]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=5, verbose=0)

In [12]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: 57.83 (42.31) MSE


# Evaluating model with standardized dataset

In [13]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: 29.44 (27.74) MSE


# A larger network

In [16]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu')) # input layer; size 13
    model.add(Dense(6, kernel_initializer='normal', activation='relu')) # hidden layer; size 6
    model.add(Dense(1, kernel_initializer='normal')) # output layer; size 1 with no activation since it's a regression problem
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [17]:
# evaluation on the larger net
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: 23.36 (26.83) MSE


# Wider network

In [18]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [19]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: 21.48 (26.30) MSE


# Using the trained models to predict price on new data

In [25]:
# Display one of the labeled cases
print(X[2])
print(Y[2])

[  2.72900000e-02   0.00000000e+00   7.07000000e+00   0.00000000e+00
   4.69000000e-01   7.18500000e+00   6.11000000e+01   4.96710000e+00
   2.00000000e+00   2.42000000e+02   1.78000000e+01   3.92830000e+02
   4.03000000e+00]
34.7


In [32]:
# Run prediction model on the above case
pipeline.fit(X,Y) # Fit model

Pipeline(steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasRegressor object at 0x121e97d68>)])

In [37]:
X_new = numpy.array([[ 0.0273, 0. , 7.07 , 0. , 0.469 , 6.421 ,
78.9 , 4.9671, 2. , 242. , 17.8 , 396.9 ,
9.14 ]])
print(X_new.shape)
print(X[2].shape)

(1, 13)
(13,)


In [35]:
pipeline.predict(X_new)

array(21.889719009399414, dtype=float32)

In [39]:
X_new = numpy.array([[2.72900000e-02, 0.00000000e+00, 7.07000000e+00, 
                      0.00000000e+00, 4.69000000e-01, 7.18500000e+00, 
                      6.11000000e+01, 4.96710000e+00, 2.00000000e+00,
                      2.42000000e+02, 1.78000000e+01, 3.92830000e+02,
                      4.03000000e+00]])
print(X_new.shape)

(1, 13)


In [41]:
Y_pred = pipeline.predict(X_new)
print("Labeled case: %2f" %Y[2])
print("Prediction: %2f" %Y_pred)

Labeled case: 34.700000
Prediction: 32.276207
